In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Pre-Processing

In [ ]:
!pip install pandas==1.0.5
!pip install tld

In [ ]:
# loading necessary packages

import os
import pandas as pd
import numpy as np
from tld import get_tld
import bz2
import json
import time
from datetime import datetime, date
import pickle
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# paths to raw data

PATH_TO_DATA = '/content/drive/MyDrive/ADAgang/data/'

PATH_TO_SPEAKER = '/content/drive/MyDrive/ADAgang/data/speaker_attributes.parquet'

PATH_TO_LABELS = '/content/drive/MyDrive/ADAgang/data/wikidata_labels_descriptions_quotebank.csv.bz2'

PATH_TO_QUOTEBANK = '/content/drive/MyDrive/ADAgang/data/Quotebank'

with open("/content/drive/MyDrive/ADAgang/generated/keywords.txt", 'r') as f:
    lines = f.read()
    KEYS = lines.splitlines()
    KEYS = [key.lower() for key in KEYS]
    KEYS.sort()

# Quotebank Data Wrangling

The first thing we will do is reduce the Quotebank data down to just the information we need since it is such a large dataset. We decide the first thing to do is to filter by a generous set of keywords that will keep all the data we will need but reduce the number of rows. Then, we will do some cleaning and store the data in a new, much smaller file.

In [ ]:
# FUNCTIONS FOR QUOTEBANK DATA WRANGLING

def filter_by_keywords(chunk, keywords):
    '''Return the rows of the dataframe only if the quotation contains any token in the keywords list.'''
    chunk_lower = chunk.copy()
    chunk_lower['quotation'] = chunk.quotation.str.lower()
    chunk = chunk[chunk_lower.quotation.str.contains('|'.join(keywords))]
    
    return chunk

def get_domains(urls):
    '''Return the domains of a given list of urls.'''
    domains = []
    for url in urls:
        domain = get_tld(url, as_object=True).domain
        domains.append(domain)
        
    return domains

def clean_chunk(chunk):
    '''Clean the dataframe.'''
    
    # remove all rows that have an unknown speaker
    chunk = chunk.replace(to_replace='None', value=np.nan).dropna(subset = ['speaker'])
    # remove all duplicates
    chunk = chunk.drop_duplicates(subset=['quoteID'], keep = 'first')
    # add column of domains extracted from urls
    chunk['domains'] = chunk.urls.apply(lambda x: get_domains(x))
    # remove unnecessary columns
    chunk = chunk.drop('phase', axis = 1)
    chunk = chunk.drop('urls', axis = 1)
    # change date to datetime format
    chunk['date'] = chunk.date.astype('datetime64').dt.date
    # set index
    chunk.set_index(chunk.quoteID, inplace=True)
    chunk = chunk.drop(['quoteID'],axis=1)
    
    return chunk

In [ ]:
# FUNCTIONs TO RUN TO PROCESS LARGE DATA

def process_data(path_to_data, path_to_out, path_to_out_sample, keys): 
    '''Filter and clean data and save to new file and create a sample of the data and save to new file.'''
    
    data = pd.read_json(path_to_data, lines=True, compression='bz2', chunksize=500000)
    start_time = time.time()
    # loop through each chunk in the data
    for chunk_idx, chunk in enumerate(data):  

        print(f'processing chunk {chunk_idx}')
        # filter the chunk by keywords and clean it
        chunk_filtered = filter_by_keywords(chunk, keys)
        chunk_cleaned = clean_chunk(chunk_filtered)
        chunk_filtered = []

        # extract a random sample (~5%) of the chunk
        sample_size = int(0.05*len(chunk))
        chunk_sample = clean_chunk(chunk.sample(sample_size))
        print(f'processed chunk {chunk_idx}')

        path = path_to_out + f'-{chunk_idx}-reduced.pck'
        path_sample = path_to_out_sample + f'-{chunk_idx}-sample.pck'
        
        chunk_cleaned.to_pickle(path, compression='bz2')
        chunk_cleaned = []
        chunk_sample.to_pickle(path_sample, compression='bz2')
        chunk_sample = []

    end_time = time.time()
    print(f'Elapsed time: {(end_time-start_time)/60} minutes.')
    
    
def process_data_year(path_to_quotebank, year, keys):
    '''Process the quotebank data for the given year.'''
    path_to_quotebank_data = path_to_quotebank + f'/quotes-{year}.json.bz2'
    path_to_out = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}'
    path_to_out_sample = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}'
    process_data(path_to_quotebank_data, path_to_out, path_to_out_sample, keys)

In [ ]:
def create_full_df(year):
    reduced_files = glob.glob("/content/drive/MyDrive/ADAgang/generated/*-reduced.pck")
    df = pd.concat([pd.read_pickle(fp, compression='bz2') for fp in reduced_files])
    path_to_reduced = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}-reduced.pkl'
    df.to_pickle(path_to_reduced, compression='bz2')

    sample_files = glob.glob("/content/drive/MyDrive/ADAgang/generated/*-sample.pck")
    df = pd.concat([pd.read_pickle(fp, compression='bz2') for fp in sample_files])
    path_to_sample = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}-sample.pkl'
    df.to_pickle(path_to_sample, compression='bz2')
    df = []
    for f in glob.glob("/content/drive/MyDrive/ADAgang/generated/*.pck"):
        os.remove(f)

### Run the following cells only once

In [ ]:
'''process_data_year(PATH_TO_QUOTEBANK, 2020, KEYS)''' # done

'process_data_year(PATH_TO_QUOTEBANK, 2020, KEYS)'

In [ ]:
'''create_full_df(2020)''' # done

'create_full_df(2020)'

In [ ]:
'''process_data_year(PATH_TO_QUOTEBANK, 2019, KEYS)''' #done

'process_data_year(PATH_TO_QUOTEBANK, 2019, KEYS)'

In [ ]:
'''create_full_df(2019)''' # done

'create_full_df(2019)'

In [ ]:
'''process_data_year(PATH_TO_QUOTEBANK, 2018, KEYS)''' # done

'process_data_year(PATH_TO_QUOTEBANK, 2018, KEYS)'

In [ ]:
'''create_full_df(2018)''' #done

'create_full_df(2018)'

In [ ]:
'''process_data_year(PATH_TO_QUOTEBANK, 2017, KEYS)''' #done

'process_data_year(PATH_TO_QUOTEBANK, 2017, KEYS)'

In [ ]:
'''create_full_df(2017)''' #done

'create_full_df(2017)'

In [ ]:
'''process_data_year(PATH_TO_QUOTEBANK, 2016, KEYS)''' #done

'process_data_year(PATH_TO_QUOTEBANK, 2016, KEYS)'

In [ ]:
'''create_full_df(2016)''' #done

'create_full_df(2016)'

In [ ]:
'''process_data_year(PATH_TO_QUOTEBANK, 2015, KEYS)''' #done

'process_data_year(PATH_TO_QUOTEBANK, 2015, KEYS)'

In [ ]:
'''create_full_df(2015)''' #done

'create_full_df(2015)'

# Prepare Dataframes for Merge

Next, we want to merge the two dataframes so we can have the speaker information as well as the quote information in the same rows. Thus, we will merge on the most likely speaker for each quote. 


First, we will check the probability distribution of the most likely speaker for each quote to see if it is worth it to keep the rest of the possible speakers.


In [ ]:
# here are some functions to help us do this:
def extract_probas(probas):
    return probas[0][1], probas[1][1]

def extract_probs1(probas):
    return probas[0][1]

def extract_probs2(probas):
    return probas[1][1]

In [ ]:
generated_data_files = glob.glob('/content/drive/MyDrive/ADAgang/generated/*.pkl')
generated_data_files

['/content/drive/MyDrive/ADAgang/generated/speaker_data_clean.pkl']

In [ ]:
def get_summary_stats(file):

    df = pd.read_pickle(file, compression='bz2')

    # we separate first and second most likely speakers into columns:

    df['prob1'] =  df.probas.apply(lambda x: extract_probs1(x))
    df['prob2'] = df.probas.apply(lambda x: extract_probs2(x))

    df = df.drop('probas',axis=1)

    #let's get some initial summary statstics:
    df['prob1'] = df.prob1.astype('float64')
    df['prob2'] = df.prob2.astype('float64')
    summary_statistics = df.describe().drop(['count','25%', '75%']).drop(['numOccurrences'],axis=1)

    return summary_statistics

In [ ]:
get_summary_stats('/content/drive/MyDrive/ADAgang/generated/intermediate/quotes-2019-reduced.pkl')

In [ ]:
get_summary_stats('/content/drive/MyDrive/ADAgang/generated/intermediate/quotes-2018-reduced.pkl')

We see that the median and mean are both quite high for the probabilities of the most likely speaker, whereas they are quite low for the second most likely speaker. Thus, we will only keep data where the most likely speaker probability is higher than 0.5, and discard all data on second and following most likely speakers.

In [ ]:
def remove_unlikely_speakers(path_to_file, path_to_out):
  
    df = pd.read_pickle(path_to_file, compression='bz2')
    df['speaker_probability'] =  df.probas.apply(lambda x: extract_probs1(x))
    df['speaker_probability'] = df.speaker_probability.astype('float64')
    df = df.drop('probas', axis=1)
    df = df[df.speaker_probability > 0.5]
    df['qids'] =  df.qids.apply(lambda x: x[0])
    df.to_pickle(path_to_out, compression='bz2')

def clean_data_year(year):
  
    path_to_reduced = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}-reduced.pkl'
    path_to_clean = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}-clean.pkl'
    remove_unlikely_speakers(path_to_reduced, path_to_clean)

    path_to_reduced_sample = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}-reduced.pkl'
    path_to_clean_sample = f'/content/drive/MyDrive/ADAgang/generated/quotes-{year}-clean.pkl'
    remove_unlikely_speakers(path_to_reduced_sample, path_to_clean_sample)

### Run the following cells only once.

In [ ]:
'''clean_data_year(2020)''' #done

'clean_data_year(2020)'

In [ ]:
'''clean_data_year(2019)''' #done

'clean_data_year(2019)'

In [ ]:
'''clean_data_year(2018)''' #done

'clean_data_year(2018)'

In [ ]:
'''clean_data_year(2017)''' #done

'clean_data_year(2017)'

In [ ]:
'''clean_data_year(2016)''' #done

'clean_data_year(2016)'

In [ ]:
'''clean_data_year(2015)''' #done

'clean_data_year(2015)'

# Speaker Data Wrangling

Loadnig the data and changing the index of the speaker dataframe to the 'id' column and making sure it is unique.

In [ ]:
'''
speaker_data_original = pd.read_parquet(PATH_TO_SPEAKER, engine = 'pyarrow')
label_descriptions = pd.read_csv(PATH_TO_LABELS, index_col = 'QID')
speaker_data = speaker_data_original.copy()
speaker_data = speaker_data.set_index('id')
'''

"\nspeaker_data_original = pd.read_parquet(PATH_TO_SPEAKER, engine = 'pyarrow')\nlabel_descriptions = pd.read_csv(PATH_TO_LABELS, index_col = 'QID')\nspeaker_data = speaker_data_original.copy()\nspeaker_data = speaker_data.set_index('id')\n"

For our project we do not foresee needing the aliases, lastrevid, type, US_congress_bio_ID, or candidacy columns so we will remove those. Also here we are changing the date_of_birth column to datetime format.

In [ ]:
'''
speaker_data = speaker_data.drop(['aliases', 'lastrevid', 'type', 'US_congress_bio_ID', 'candidacy'],axis = 1)
speaker_data['date_of_birth'] = speaker_data.date_of_birth.apply(lambda x: x[0][1:11] if x is not None else None)
speaker_data['date_of_birth'] = pd.to_datetime(speaker_data.date_of_birth, errors = 'coerce')
'''

"\nspeaker_data = speaker_data.drop(['aliases', 'lastrevid', 'type', 'US_congress_bio_ID', 'candidacy'],axis = 1)\nspeaker_data['date_of_birth'] = speaker_data.date_of_birth.apply(lambda x: x[0][1:11] if x is not None else None)\nspeaker_data['date_of_birth'] = pd.to_datetime(speaker_data.date_of_birth, errors = 'coerce')\n"

The rest of the columns will be useful for our analysis. However, they are not very interpretable. Thus, we will go through each column and replace ids with meaningful labels we will need.

First, we will work with the gender column since we expect fewer unique values here than in other columns. Let's see what possible labels we can get by searching 'gender' in the label decriptions dataframe.

In [ ]:
'''label_descriptions[label_descriptions.Description.str.contains('gender', na=False)]'''

"label_descriptions[label_descriptions.Description.str.contains('gender', na=False)]"

Although we expect most of the genders of speakers to be male/female, we will keep five labels for gender: male, female, trans male, trans female, and other. This way, we avoid erasure of gender diversity  while still keeping simple labels. However, we will only allow one label per speaker.

In [ ]:
def determine_gender(gender_id):
    
    # determined from 'label_descriptions[label_descriptions.QID.isin(unique_genders)]'
    male, female, trans_male, trans_female = ['Q6581097'], ['Q6581072', 'Q15145779'], ['Q2449503'], ['Q1052281']

    if gender_id is None:
        gender = 'None'
    elif gender_id[0] in trans_female:
        gender = 'trans female'
    elif gender_id[0] in trans_male:
        gender = 'trans male'
    elif gender_id[0] in female:
        gender = 'female'
    elif gender_id[0] in male:
        gender = 'male'
    else:
        gender = 'other'
    
    return gender


Next, we will go through the 'party' column. Since our analysis is on Black Lives Matter, an movement in the United States, only American political parties are relevant. Thus, we will label each speaker with a party of "Republican," "Democrat," or "Other."

In [ ]:
'''
# getting all political parties from label_descriptions
parties = label_descriptions[label_descriptions.Description.str.contains('political party', na=False)]

# of these, getting just parties that are American
parties_us = parties[parties.Description.str.contains('|'.join(['U.S', 'United States']), na=False)]

# we really only want the republican or democrat labels
parties_us[parties_us.Label.str.contains('|'.join(['Republican', 'Democrat']))]

# separate into groups
democrats = parties_us[parties_us.Label.str.contains('Democrat')]
republicans = parties_us[parties_us.Label.str.contains('Republican')]
'''

"\n# getting all political parties from label_descriptions\nparties = label_descriptions[label_descriptions.Description.str.contains('political party', na=False)]\n\n# of these, getting just parties that are American\nparties_us = parties[parties.Description.str.contains('|'.join(['U.S', 'United States']), na=False)]\n\n# we really only want the republican or democrat labels\nparties_us[parties_us.Label.str.contains('|'.join(['Republican', 'Democrat']))]\n\n# separate into groups\ndemocrats = parties_us[parties_us.Label.str.contains('Democrat')]\nrepublicans = parties_us[parties_us.Label.str.contains('Republican')]\n"

Here we manually remove the unrelated parties.

In [ ]:
'''democrats'''

'democrats'

In [ ]:
'''
remove = ['Social Democratic Federation', 
          'Social Democratic League of America',
          'Liberal Democratic Party of the Soviet Union',
          'Minnesota Democratic–Farmer–Labor Party']

democrats = democrats[~(democrats.Label.isin(remove))]
democrat_ids = democrats.index
democrat_ids
'''

"\nremove = ['Social Democratic Federation', \n          'Social Democratic League of America',\n          'Liberal Democratic Party of the Soviet Union',\n          'Minnesota Democratic–Farmer–Labor Party']\n\ndemocrats = democrats[~(democrats.Label.isin(remove))]\ndemocrat_ids = democrats.index\ndemocrat_ids\n"

In [ ]:
'''republicans'''

'republicans'

In [ ]:
'''
republicans = republicans[~(republicans.Label.isin(['Silver Republican Party']))]
republican_ids = republicans.index
republican_ids
'''

"\nrepublicans = republicans[~(republicans.Label.isin(['Silver Republican Party']))]\nrepublican_ids = republicans.index\nrepublican_ids\n"

In [ ]:
def determine_party(party_id):
    
    democrats = ['Q5255714', 'Q10594775', 'Q16988710', 'Q29552', 'Q4708523', 'Q6781300','Q5255727', 'Q7707623']
    republicans = ['Q7314666', 'Q29468']
    
    if party_id is None:
        party = None
    else:
        if party_id[0] in republicans:
            party = 'Republican'
        elif party_id[0] in democrats:
            party = 'Democrat'
        else:
            party = 'Other'
            
    return party
        

In [ ]:
'''
speaker_data['gender'] = speaker_data.gender.apply(lambda x: determine_gender(x))
speaker_data['party'] = speaker_data.party.apply(lambda x: determine_party(x))
'''

"\nspeaker_data['gender'] = speaker_data.gender.apply(lambda x: determine_gender(x))\nspeaker_data['party'] = speaker_data.party.apply(lambda x: determine_party(x))\n"

In [ ]:
label_descriptions = []

Now we will merge the dataframes to have all our quote information and the speaker information together.

In [ ]:
def merge_dataframes(df, speaker_data, file_path):
    merged_df = pd.merge(df, speaker_data, left_on='qids', right_index=True, how = 'left')
    merged_df.to_pickle(file_path, compression='bz2')

In [ ]:
label_descriptions = []

In [ ]:
'''
df = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2020-clean.pkl', compression = 'bz2')
merged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2020-data-merged.pkl.bz2'
merge_dataframes(df, speaker_data, merged_path_to_out)
'''
#done

"\ndf = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2020-clean.pkl', compression = 'bz2')\nmerged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2020-data-merged.pkl.bz2'\nmerge_dataframes(df, speaker_data, merged_path_to_out)\n"

In [ ]:
'''
df = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2019-clean.pkl', compression = 'bz2')
merged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2019-data-merged.pkl.bz2'
merge_dataframes(df, speaker_data, merged_path_to_out)
''' 
#done

"\ndf = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2019-clean.pkl', compression = 'bz2')\nmerged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2019-data-merged.pkl.bz2'\nmerge_dataframes(df, speaker_data, merged_path_to_out)\n"

In [ ]:
'''
df = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2018-clean.pkl', compression = 'bz2')
merged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2018-data-merged.pkl.bz2'
merge_dataframes(df, speaker_data, merged_path_to_out)
'''
#done

"\ndf = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2018-clean.pkl', compression = 'bz2')\nmerged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2018-data-merged.pkl.bz2'\nmerge_dataframes(df, speaker_data, merged_path_to_out)\n"

In [ ]:
'''
df = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2017-clean.pkl', compression = 'bz2')
merged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2017-data-merged.pkl.bz2'
merge_dataframes(df, speaker_data, merged_path_to_out)
'''
#done

"\ndf = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2017-clean.pkl', compression = 'bz2')\nmerged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2017-data-merged.pkl.bz2'\nmerge_dataframes(df, speaker_data, merged_path_to_out)\n"

In [ ]:
'''
df = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2016-clean.pkl', compression = 'bz2')
merged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2016-data-merged.pkl.bz2'
merge_dataframes(df, speaker_data, merged_path_to_out)
'''
#done

"\ndf = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2016-clean.pkl', compression = 'bz2')\nmerged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2016-data-merged.pkl.bz2'\nmerge_dataframes(df, speaker_data, merged_path_to_out)\n"

In [ ]:
'''
df = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2015-clean.pkl', compression = 'bz2')
merged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2015-data-merged.pkl.bz2'
merge_dataframes(df, speaker_data, merged_path_to_out)
'''
#done

"\ndf = pd.read_pickle('/content/drive/MyDrive/ADAgang/generated/quotes-2015-clean.pkl', compression = 'bz2')\nmerged_path_to_out = '/content/drive/MyDrive/ADAgang/generated/2015-data-merged.pkl.bz2'\nmerge_dataframes(df, speaker_data, merged_path_to_out)\n"

In [ ]:

path = '/content/drive/MyDrive/ADAgang/generated/'

df_2015 = pd.read_pickle(path + '2015-data-merged.pkl.bz2', compression='bz2')
df_2016 = pd.read_pickle(path + '2016-data-merged.pkl.bz2', compression='bz2')
df_2017 = pd.read_pickle(path + '2017-data-merged.pkl.bz2', compression='bz2')
df_2018 = pd.read_pickle(path + '2018-data-merged.pkl.bz2', compression='bz2')
df_2019 = pd.read_pickle(path + '2019-data-merged.pkl.bz2', compression='bz2')
df_2020 = pd.read_pickle(path + '2020-data-merged.pkl.bz2', compression='bz2')
df_total = pd.concat([df_2015,df_2016,df_2017,df_2018,df_2019,df_2020])


Now we just need to make the remaining labels interpretable on the final dataset we will be using going forward.

In [ ]:
label_descriptions = pd.read_csv(PATH_TO_LABELS, index_col = 'QID')

In [ ]:
def get_label(x):
    labels = []
    if len(x) == 1:
        try:
            labels = label_descriptions.loc[x[0]].Label
        except KeyError:
    labels = None
    else:
        for i in x:
            try:
                label = label_descriptions.loc[i].Label
                labels.append(label)
            except KeyError:
                pass
    return labels

In [ ]:
df_total.drop('label', axis = 1)
df_total['date'] = df_total.date.astype('datetime64').dt.date
df_total = df_total.where(pd.notnull(df_total), None)
df_total['nationality'] = df_total.nationality.apply(lambda x: get_label(x) if x is not None else None)
df_total['ethnic_group'] = df_total.ethnic_group.apply(lambda x: get_label(x) if x is not None else None)
df_total['occupation'] = df_total.occupation.apply(lambda x: get_label(x) if x is not None else None)
df_total['religion'] = df_total.religion.apply(lambda x: get_label(x) if x is not None else None)
df_total['academic_degree'] = df_total.academic_degree.apply(lambda x: get_label(x) if x is not None else None)


In [ ]:
df_total.to_pickle('/content/drive/MyDrive/ADAgang/generated/total-data-merged.pkl.bz2', compression='bz2')